In [1]:
#Bowerbird null model (all males are guarders)

#Key
#Notes- Paramters contain underscores, while functions don't
#SB: Stay at bower
#FG: Foraging
#a=-1: denotes a staying at bower action on ticket
#a=-2: denotes a foraging action on ticket
#a=-3: denotes a female visiting action on a ticket
#networkwriter: generates network where everybird is connected
#addtotimeline: adds tickets to timeline
#travel_times_linear_p: generates a matrix that contains the probabilities of travelling to a given a bower 
#if the relationship between distance and travel preference is linear


import math
import random
import numpy as np


# makes a ticket
def ticketgenerator(t, o, a):
    ticket={
        'time': t,
        'owner': o,
        'action': a,
    }
    return ticket;

#writes the edges to a network of n birds where everyone is connected to everyone else 
def networkwriter(n):
    connect=[0]*n
    for i in range(0, n): 
        a=list(range(n)) #generating a list that goes from 0 to n-1
        a.remove(i) #individual removes themselves from the network
        connect[i]=a #corresponds to the list of people person i is linked to
    return connect; #returns a list of lists with all the connections in the network

# function for determining the next time based on our rate parameters
def nexttau(rateParameter):
    taus=-math.log(1.0 - random.random()) / rateParameter #math.log is in base e, and random.random picks a number from range 0.0 to 1.
    return taus #generates n-length list, starting at index 0

#adds new tickets to timeline -- Stefano, any more efficient suggestions for keeping timeline in order?
def addtotimeline(tic, timeline):
    if not timeline:
        timeline.append(tic)
    else:
        ind=len(timeline)-1
        end=0
        while (tic['time']<timeline[ind]['time'] and end==0): #moves backwards until it finds where to place the ticket based on the listed times
            ind=ind-1 
            if(ind<0):
                end=1
        ind=ind+1
        timeline.insert(ind, tic)
    

def SBtickethandler (SB_tic, timeline,SB_param, t_max):
    ow=SB_tic['owner'] 
    t=SB_tic['time']
    FG_time=nexttau(SB_param)+t #We use SB_param because the time represents when the bird leaves the bower
    if FG_time<t_max:
        FG_tic=ticketgenerator(FG_time,ow,-2) #-2 denotes foraging action
        addtotimeline(FG_tic,timeline)

def FGtickethandler (FG_tic, timeline,FG_param, t_max):
    ow=FG_tic['owner'] 
    t=FG_tic['time']
    SB_time=nexttau(FG_param)+t #We use FG_param because the time represents when the bird returns to the bower
    if SB_time<t_max:
        SB_tic=ticketgenerator(SB_time,ow,-1) #-1 denotes staying at bower
        addtotimeline(SB_tic,timeline)
    
def FVtickethandler (FV_tic, timeline, FV_param, t_max, bower_states, male_states, fitness_states, nodes, female_preferences, travel_times):
    ow=FV_tic['owner']
    t=FV_tic['time']
    if bower_states[ow]==1 and male_states[ow]==1: #if the bower is intact and the male is present
        fitness_states[ow]=fitness_states[ow]+1 #assumption: female always mates if bower is intact and male present
        new_FV_ow=np.random.choice(list(range(nodes)))
        new_FV_time=t+nexttime(FV_param)
        if new_FV_time<t_max:
            new_FV_tic=ticketgenerator(new_FV_time, new_FV_ow, -3)
            addtotimeline(new_FV_ticket, timeline)
    else: #if female does not successfully mate
        new_FV_ow=np.random.choice(list(range(nodes)), p=female_preferences[ow]) #choose a male based on preference (a function of distance)
        new_FV_time=t+travel_times[ow][new_FV_ow] #she goes directly to this male
        if new_FV_time<t_max:
            new_FV_tic=ticketgenerator(new_FV_time, new_FV_ow, -3)
            addtotimeline(new_FV_tic, timeline)
        
    
#FOR FUTURE: Write a function that generates a probability network of n bowerbirds 

#for now, we write a simple example network with 4 bowers forming a square
nodes=4

#HOW TO REPRESENT INDIVIDUAL'S DIST FROM SELF? nan?
distances = np.array([[150.0]*nodes]*nodes) #(dist in m)initialize a nodes-by-nodes matrix (1st nrows, 2nd ncols)
for i in range(4):
    distances[i][i]=9999.0 #or math.inf but that makes the matrix disp ugly
for i in range(2):
    distances[i][i+2]=((2*(150**2)))**(1/2)
    distances[i+2][i]=((2*(150**2)))**(1/2)
print(distances)

bird_speed=12 #m/s
travel_times=np.array([[0.0]*nodes]*nodes)
for i in range(nodes):
    for j in range(nodes):
        travel_times[i][j]=distances[i][j]/bird_speed
print(travel_times) #right now the numbers on the diagonals aren't accurate, but we can't have the bird choosing to go to it's own node

 

#will write female preference based on cumulative exponential decay (lambda=100/99)
female_preferences=np.array([[0.]*nodes]*nodes)

for i in range(nodes):
    for j in range(nodes):
        female_preferences[i][j]=math.exp(-100/99*distances[i][j])        
    female_preferences[i]=female_preferences[i]/sum(female_preferences[i])
print(female_preferences)

    


[[9999.          150.          212.13203436  150.        ]
 [ 150.         9999.          150.          212.13203436]
 [ 212.13203436  150.         9999.          150.        ]
 [ 150.          212.13203436  150.         9999.        ]]
[[833.25        12.5         17.67766953  12.5       ]
 [ 12.5        833.25        12.5         17.67766953]
 [ 17.67766953  12.5        833.25        12.5       ]
 [ 12.5         17.67766953  12.5        833.25      ]]
[[0.0000000e+00 5.0000000e-01 2.7720989e-28 5.0000000e-01]
 [5.0000000e-01 0.0000000e+00 5.0000000e-01 2.7720989e-28]
 [2.7720989e-28 5.0000000e-01 0.0000000e+00 5.0000000e-01]
 [5.0000000e-01 2.7720989e-28 5.0000000e-01 0.0000000e+00]]


In [2]:
#Create travel_time__linear_p
travel_times_linear_p=np.array([[0.0]*nodes]*nodes)
for i in range(nodes):
    for j in range(nodes):
        if i==j: #if the row and the column index are the equal
            travel_times[i][j]=0 #There is 0 probability they will travel to their own bower when choosing to fly to another bower
        else: 
            if i==0: #if on the first row of the probability matrix
                travel_times_linear_p[i][j]=travel_times[i][1]/travel_times[i][j] #skip the first element in the row (because it equals 0) and compare probabilities of other elements in the row based on travel distance (negative linear relationship btwn distance and prob)
            else:
                travel_times_linear_p[i][j]=travel_times[i][0]/travel_times[i][j] #use the first element in a row and compare probabilities of other elements in the row based on distance (negative linear relationship btwn distance and prob)
    travel_times_linear_p[i]=travel_times_linear_p[i]/sum(travel_times_linear_p[i]) #normalize the probabilities so they add up to 1
print(travel_times_linear_p)




[[0.         0.36939806 0.26120387 0.36939806]
 [0.36939806 0.         0.36939806 0.26120387]
 [0.26120387 0.36939806 0.         0.36939806]
 [0.36939806 0.26120387 0.36939806 0.        ]]


In [3]:
#just a test

    

first_tic=ticketgenerator(0, 2, -3)
timeline=[]
addtotimeline(first_tic, timeline)
FV_param=3 #totally random
t_max=1000
bower_states=[1, 1, 1, 1]
male_states=[1,1,0,1]
fitness_states=[0,0,0,0]
nodes=4
female_preferences=travel_times_linear_p

FVtickethandler(first_tic, timeline, FV_param, t_max, bower_states, male_states, fitness_states, nodes, female_preferences, travel_times)




In [4]:
print(timeline)
print(fitness_states)

[{'time': 0, 'owner': 2, 'action': -3}, {'time': 17.67766952966369, 'owner': 0, 'action': -3}]
[0, 0, 0, 0]


In [5]:
#adds new tickets to timeline -- Stefano, any more efficient suggestions for keeping timeline in order?
def addtotimeline(tic, timeline):
    if not timeline:
        timeline.append(tic)
    else:
        ind=len(timeline)-1
        end=0
        while (tic['time']<timeline[ind]['time'] and end==0): #moves backwards until it finds where to place the ticket based on the listed times
            ind=ind-1 
            if(ind<0):
                end=1
        ind=ind+1
        timeline.insert(ind, tic)
    

first_tic=ticketgenerator(0, 2, -3)
timeline2=[]
addtotimeline(first_tic,timeline2)
print(timeline2)

[{'time': 0, 'owner': 2, 'action': -3}]
